In [1]:
from datetime import datetime
import matplotlib.pyplot as plt
import psycopg2
import pandas as pd
import geopandas as gpd
import numpy as np
import json
from sqlalchemy import create_engine, text
from tqdm import tqdm
from geopy.distance import geodesic
from pymongo import MongoClient
from shapely.geometry import Point, LineString

client = MongoClient('localhost', 27017)


# Define database connection parameters
database_connection = {
    'drivername': 'postgresql',
    'username': 'postgres',
    'password': 'postgres',
    'host': 'localhost',
    'port': '5432',
    'database': 'vic_db',
}


# A psycopg2 connection and cursor
conn = psycopg2.connect(user=database_connection['username'],
                        password=database_connection['password'],
                        host=database_connection['host'],
                        port=database_connection['port'],
                        database=database_connection['database'])
conn.autocommit = True
cursor = conn.cursor()

# Create a SQLAlchemy engine
engine = create_engine('postgresql://%(username)s:%(password)s@%(host)s/%(database)s' % database_connection, isolation_level="AUTOCOMMIT")
conn_alchemy = engine.connect()


In [9]:

sql = '''
SELECT *
FROM INFORMATION_SCHEMA.COLUMNS
WHERE TABLE_NAME = 'tr_road_all' AND TABLE_SCHEMA = 'vmtrans';
'''

cursor.execute(sql)
data = cursor.fetchall()
columns = [c[0] for c in cursor.description]
df = pd.DataFrame(data, columns=columns)


In [18]:
['integer', 'double precision', 'character varying', 'date',
       'USER-DEFINED']
df[df['data_type'] == 'integer']['column_name'].values

array(['gid', 'class_code'], dtype=object)

In [22]:
# Cast ufi of table tr_road_all from a double to an integer then to a string
['ufi', 'pfi', 'named_feature_id', 'from_ufi', 'to_ufi',
       'feature_quality_id', 'task_id', 'superceded_pfi']
double_to_int = [
    'ufi', 'pfi', 'named_feature_id', 'from_ufi', 'to_ufi',
    'feature_quality_id', 'task_id', 'superceded_pfi'
]
sql_list = [
    f"ALTER TABLE vmtrans.tr_road_all ALTER COLUMN {c} TYPE INTEGER USING {c}::INTEGER;" for c in double_to_int
]
print("set PGPASSWORD=postgres")
for sql in sql_list:
    print(f"psql -U postgres -h localhost -p 5432 -d vic_db -c '{sql}'")
    # print(cursor.statusmessage, cursor.fetchall())

set PGPASSWORD=postgres
psql -U postgres -h localhost -p 5432 -d vic_db -c 'ALTER TABLE vmtrans.tr_road_all ALTER COLUMN ufi TYPE INTEGER USING ufi::INTEGER;'
psql -U postgres -h localhost -p 5432 -d vic_db -c 'ALTER TABLE vmtrans.tr_road_all ALTER COLUMN pfi TYPE INTEGER USING pfi::INTEGER;'
psql -U postgres -h localhost -p 5432 -d vic_db -c 'ALTER TABLE vmtrans.tr_road_all ALTER COLUMN named_feature_id TYPE INTEGER USING named_feature_id::INTEGER;'
psql -U postgres -h localhost -p 5432 -d vic_db -c 'ALTER TABLE vmtrans.tr_road_all ALTER COLUMN from_ufi TYPE INTEGER USING from_ufi::INTEGER;'
psql -U postgres -h localhost -p 5432 -d vic_db -c 'ALTER TABLE vmtrans.tr_road_all ALTER COLUMN to_ufi TYPE INTEGER USING to_ufi::INTEGER;'
psql -U postgres -h localhost -p 5432 -d vic_db -c 'ALTER TABLE vmtrans.tr_road_all ALTER COLUMN feature_quality_id TYPE INTEGER USING feature_quality_id::INTEGER;'
psql -U postgres -h localhost -p 5432 -d vic_db -c 'ALTER TABLE vmtrans.tr_road_all ALTER COLU